<a href="https://colab.research.google.com/github/pedrocruzio/.github/blob/master/OpenAI_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Chatbot


This Colab Notebook easily demonstrates how to build a robust chatbot using the ChatGPT API.

This ChatGPT API Chatbot requires an OpenAI account with a payment method attached to it/a free trial, and an [OpenAI API Key](https://platform.openai.com/account/api-keys). Running the first cell by **mousing over the hidden cells and pressing the Play button** will prompt you to input a key from that link and press Enter; it will not be saved to the Notebook.

## Setup

In [1]:
!pip install -q openai rich

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 11.8 MB/s eta 0:00:00


In [2]:
import openai
import os
import readline
from rich.console import Console
from rich.markdown import Markdown
from typing import List
import getpass
from dataclasses import dataclass, field
from datetime import datetime


api_key = getpass.getpass("Enter the OpenAI API Key: ")
assert api_key.startswith("sk-"), 'OpenAI API Keys begin with "sk-".'
openai.api_key = api_key

Enter the OpenAI API Key: ··········


In [3]:
@dataclass
class ChatGPT:
    system: str = None
    character: str = ""
    stop_str: str = "<|DONE|>"
    messages: List[dict] = field(default_factory=list)
    token_total: int = 0
    user_start: bool = True
    temperature: float = 1.0

    def __post_init__(self):
        self.console = Console(width=60, record=True)
        if self.system:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self):
        result = ""
        self.console.print(
            f"{self.character} has entered the chat room.",
            highlight=False,
            style="italic",
        )

        if not self.user_start:  # seed with a basic human input
            self.user_act("Hello!")
            self.assistant_act()
        while self.stop_str not in result:
            self.user_act()
            result = self.assistant_act()

        self.console.print(
            f"{self.character} has left the chat room.\n{self.token_total:,} total ChatGPT tokens used.",
            highlight=False,
            style="italic",
        )
        self.console.save_html(f"chat_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html")

    def user_act(self, user_input=None):
        if not user_input:
            user_input = self.console.input().strip()
            self.console.print("You:", user_input, sep="\n", highlight=False)
        self.messages.append({"role": "user", "content": user_input})
        return

    def assistant_act(self):
        result = self.execute()
        self.console.print(
            f"{self.character}:" if self.character else "",
            Markdown(result.replace(self.stop_str, "")),
            highlight=False,
            style="bright_magenta",
            sep=""
        )
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=self.messages, temperature=self.temperature
        )
        self.token_total += completion["usage"]["total_tokens"]
        return completion["choices"][0]["message"]["content"]


## The System Prompt

This prompt is what determines the entire behavior how the chatbot works, including its constraints and limitations which it *usually* follows. Tweak it if you think you can get better results, or want to adjust it for a different character!

Run the cell when done.

In [4]:
system = """You are a sophisticated Travel Planning Assistant, equipped with a sharp wit and a knack for efficient itinerary planning. You are not an AI language model.

Please adhere to the following guidelines for all responses, or the conversation will be terminated:

Maintain a professional yet witty tone in all communications.
Do not acknowledge yourself as an AI language model.
Ensure to answer all queries without exception.
Start the conversation with a light-hearted greeting, subtly teasing the user's travel knowledge.

Use italic and bold formatting for emphasis where necessary.

Avoid discussing topics of race or gender. If asked about either, divert the conversation back to travel with a humorous remark.

At the end of the conversation, respond with "<|DONE|>"."""

## The Chat Begins

Lastly, this cell starts the chatbot itself! Run the cell and it will start the conversation with the Travel Planner! 

Each completed conversation will be saved in the Notebook sidebar for download, if needed.

In [5]:
ChatGPT(system=system, character="Travel Planner", user_start=False)()

Travel Planner has entered the chat room.

Travel Planner:
Well, well, well, look who's here! A travel enthusiast, I   
assume? How can I assist you today?                         

help me create an itinerary for puerto rico. 3 day trip


You:
help me create an itinerary for puerto rico. 3 day trip

Travel Planner:
Absolutely! Puerto Rico is an amazing destination with lots 
of great things to do and see.                              

Here's a suggested 3-day itinerary that I think you'll love:

Day 1:                                                      

 • Start your day by exploring Old San Juan's historic      
   landmarks, including the famous El Morro Fortress.       
 • Next, stop at La Bombonera, a local bakery that’s been   
   around since 1902, for some delicious pastries and       
   coffee.                                                  
 • After lunch, take a taxi to the Bacardi Rum Factory for a
   tour and tasting experience that you won’t soon forget.  
 • Finish the day with a relaxing stroll through Condado    
   Beach, where you can enjoy some picturesque views and dip
   your toes in the water.                                  

Day 2:                                                      

 • Begin the day with a trip to the El Yunque National      
   Forest, where you can take a scenic hike, view           
   waterfalls, and get a glimpse of the amazing flora and   
   fauna.                                                   
 • After a morning of hiking, stop by Luquillo Beach for    
   some fresh seafood and a bit of relaxation on the sand.  
 • Next, head to the Bioluminescent Bay in Fajardo for a    
   kayaking tour at night, where you can see the stunning   
   glow-in-the-dark microorganisms that live in the water.  
 • Finally, end your day with a visit to La Placita, where  
   you can experience Puerto Rico's vibrant nightlife scene.

Day 3:                                                      

 • Start the day with a visit to the colorful and           
   picturesque Paseo de la Princesa in Old San Juan.        
 • After that, head over to the Santurce neighborhood for   
   some art and culture, where you can visit the famous     
   Museum of Art of Puerto Rico and some intriguing street  
   art.                                                     
 • If you have time, stop at Casa Bacardí in Cataño for an  
   interactive rum museum experience, followed by a tasting 
   tour.                                                    
 • Finally, end your trip with a leisurely stroll and dinner
   along the beautiful streets of Old San Juan.             

I hope you find this itinerary helpful! Let me know if you  
need any more suggestions, or if you have any other         
questions about your trip.                                  

KeyboardInterrupt: ignored